In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



In [3]:
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents) # should I only take the first? which is the most relevant?

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [5]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [6]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [7]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [12]:
res_dict = {}
for q in questions:
    res = chain(
        {"question": q},
        return_only_outputs=True,
    )
    res_dict[q] = res["answer"].strip()

In [14]:
for q in questions:
    print(res_dict[q])

Atelectasis is the partial collapse or closure of a lung resulting in reduced or absent gas exchange. It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down to little or no volume. It is often referred to informally as a collapsed lung, although more accurately it usually involves only a partial collapse. The term uses combining forms of atel- + ectasis, from Greek: ἀτελής, "incomplete" + ἔκτασις, "extension". It is a very common finding in chest X-rays and other radiological studies, and may be caused by normal exhalation or by various medical conditions. Although frequently described as a collapse of lung tissue, atelectasis is not synonymous with a pneumothorax, which is a more specific condition that can cause atelectasis. Acute atelectasis may occur as a post-operative complication or as a result of surfactant deficiency. In premature babies, this leads to infant respiratory distress syndrome. (
The symptoms associated wi

In [9]:
raise StopIteration()

StopIteration: 

In [ ]:
res = chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange. It is usually unilateral, affecting part or all of one lung. It is governed by English law.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis, 28-pl'}

In [ ]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, it is a common misconception that atelectasis causes fever. There is no clinical evidence supporting this speculation. Atelectasis is often a common finding in chest X-rays and other radiological studies and may be caused by normal exhalation or various medical conditions. It is usually unilateral, affecting part or all of one lung. The most common cause of atelectasis is post-surgical atelectasis, which develops in 75-90% of people undergoing general anesthesia for a surgical procedure. Other causes include pulmonary tuberculosis, smoking, and age-related factors. The diagnosis of atelectasis is generally visible on chest X-ray, and treatment depends on the underlying cause. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasi

In [ ]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by normal exhalation or by various medical conditions such as post-operative complications, surfactant deficiency, and infant respiratory distress syndrome. It can also be caused by factors such as bronchial obstruction, poor surfactant spreading, and suction. (',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis)'}

In [ ]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'For diagnosing Atelectasis, radiologists need to look for signs such as lung opacification, loss of lung volume, displacement of interlobar fissures, and mobile structures within the thorax. These features are generally visible on a chest X-ray. Additionally, clinical symptoms such as cough, chest pain, breathing difficulty, low oxygen saturation, pleural effusion, cyanosis, and increased heart rate may also be present. A CT scan or bronchoscopy may be necessary if the cause of atelectasis is not clinically apparent. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [ ]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiographic features of atelectasis include the collapse or closure of a lung resulting in reduced or absent gas exchange, unilateral involvement of part or all of one lung, and deflation of the alveoli. It is often seen in chest X-rays and other radiological studies. Atelectasis is not synonymous with pneumothorax, although it can cause atelectasis. It can be caused by normal exhalation or various medical conditions. Acute atelectasis may occur as a post-operative complication or due to surfactant deficiency. In premature babies, it can lead to infant respiratory distress syndrome. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}